In [1]:
import os
import re
import sqlite3
import pandas as pd
import json
import openai
import argparse
from io import StringIO
from langchain.schema import ChatMessage
from langchain.chat_models import ChatOpenAI

# from langchain_experimental.llms import ChatLlamaAPI
# from llamaapi import LlamaAPI
# Replace 'Your_API_Token' with your actual API token
# llama = LlamaAPI('Your_API_Token')

from langchain.schema import AIMessage, HumanMessage, SystemMessage

conn = sqlite3.connect("test_db", check_same_thread=False)

In [ ]:
def update_output(contents, filename):
    if contents is not None:
        content_type, content_string = contents.split(",")
        decoded = base64.b64decode(content_string)

        if "csv" in filename:
            df = pd.read_csv(io.StringIO(decoded.decode("utf-8")))
        elif "xls" in filename:
            df = pd.read_excel(io.BytesIO(decoded))
        else:
            return html.Div(["There was an error processing this file."])
        sampled_df = df.sample(5)
        return df, df.head(), sampled_df


In [40]:
import pandas as pd




In [75]:
import pandas as pd
from collections import Counter
import datetime

def sample_dataframe(df):
    num_rows = len(df)
    # Define the penalty function based on the size of the DataFrame
    if num_rows < 1000:
        sample_size = 10
    elif num_rows < 10000:
        sample_size = 20
    else:
        sample_size = 30

    return df.sample(n=sample_size, random_state=5)

# Function to infer the most common type for a column
def infer_common_type(series):
    # Count the occurrences of each type
    type_counts = Counter()
    for item in series:
        if isinstance(item, bool):
            type_counts[bool] += 1
        elif isinstance(item, int):
            type_counts[int] += 1
        elif isinstance(item, float):
            type_counts[float] += 1
        elif isinstance(item, str):
            # List of common datetime formats
            date_formats = [
                '%Y-%m-%d', 
                '%m/%d/%Y', 
                '%d/%m/%Y', 
                '%Y-%m-%d %H:%M:%S',
                '%m/%d/%Y %H:%M:%S',
                '%d/%m/%Y %H:%M:%S',
            ]
            
            date_detected = False
            for date_format in date_formats:
                try:
                    datetime.datetime.strptime(item, date_format)
                    type_counts[datetime.datetime] += 1
                    date_detected = True
                    break
                except ValueError:
                    continue
            
            if not date_detected:
                type_counts[str] += 1
    
    return type_counts.most_common(1)[0][0]

def coerce_df_datatypes(df):
    sample_df = sample_dataframe(df)
    print('sample df size', sample_df.shape)
    # Infer the most common data type for each column
    common_data_types = {column: infer_common_type(sample_df[column]) for column in sample_df.columns}

    # Coerce the DataFrame based on the inferred types
    for column, common_type in common_data_types.items():
        if common_type == int:
            df[column] = pd.to_numeric(df[column], errors='coerce')
        elif common_type == float:
            df[column] = pd.to_numeric(df[column], errors='coerce')
        elif common_type == str:
            df[column] = df[column].astype(str)
        elif common_type == datetime.datetime:
            df[column] = pd.to_datetime(df[column], errors='coerce')
        elif common_type == bool:
            df[column] = df[column].astype(bool)

    # You can now continue working with the coerced DataFrame
    print(df.dtypes)
    return df


# Read the CSV file
file_path = './data/cars.csv'
file_path = './data/dummy_data.csv'
loaded_df = pd.read_csv(file_path)
#col_fix_df = coerce_df_datatypes(loaded_df)
print(loaded_df.dtypes)
loaded_df.head()

Name        object
Age          int64
Gender      object
Income     float64
 ozip      float64
 dzip      float64
 signup     object
dtype: object


,Name,Age,Gender,Income,ozip,dzip,signup
0,Alice,25,F,50000.0,55345.0,56002.0,NaN
1,Bob,-5,M,55000.0,56001.0,55345.0,11-22-2000
2,Carol,30,F,NaN,NaN,NaN,11-22-2011
3,David,128,M,60000.0,NaN,NaN,1-22-2000
4,Eve,29,F,65000.0,NaN,NaN,2023-01-01


In [74]:
sample_df = sample_dataframe(loaded_df)
print(sample_df.dtypes)
sample_df

Name               object
Age                 int64
Gender             object
Income            float64
 ozip             float64
 dzip             float64
 signup    datetime64[ns]
dtype: object


,Name,Age,Gender,Income,ozip,dzip,signup
2,Carol,30,F,NaN,NaN,NaN,2011-11-22
5,Frank,20,M,NaN,NaN,NaN,NaT
17,Rita,50,F,NaN,NaN,NaN,NaT
19,Tina,35,F,40000.0,NaN,NaN,NaT
12,Mona,-10,F,95000.0,NaN,NaN,NaT
1,Bob,-5,M,55000.0,56001.0,55345.0,2000-11-22
11,Larry,15,M,90000.0,NaN,NaN,NaT
10,Kathy,44,F,85000.0,NaN,NaN,NaT
13,Nick,66,M,100000.0,NaN,NaN,NaT
18,Sam,60,M,120000.0,NaN,NaN,NaT


In [102]:
csv_data = sample_df.to_csv(index=False)
csv_dtypes = sample_df.dtypes

In [2]:
def sql_chat(prompt: str, chat_history: list, instructions_prompt: str):
    chat_history.append(ChatMessage(role="assistant", content=instructions_prompt))
    chat_history.append(ChatMessage(role="user", content=prompt))

    openai.api_key = os.getenv("OPENAI_API_KEY")
    chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    response = chat(chat_history)
    chat_history.append(ChatMessage(role="assistant", content=response.content))

SyntaxError: incomplete input (2155732873.py, line 8)

In [127]:
table_name = 'dummy_table'
instructions_prompt = f"""You are a data analyst.
    Your job is to create a DESCRIPTION for each FIELD_NAME from data sample below provide the response in the OUTPUT_FORMAT provided.
    
    Here is the data to create each FIELD_NAME DESCRIPTION {csv_data}.
    
    The TABLE_NAME is {table_name}.
    
    The DATA_TYPE are in {csv_dtypes}.  These DATA_TYPES will need to be converted to sqlite datatypes.
    
    Include the Headers on top the response.
    
    OUTPUT_FORMAT
    TABLE_NAME, FIELD_NAME, DESCRIPTION, DATA_TYPE
    """

In [128]:
chat_history = []
chat_history.append(ChatMessage(role="assistant", content=instructions_prompt))
openai.api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
response = chat(chat_history)

In [132]:
rsp = response.content
df = pd.read_csv(StringIO(rsp.strip()), sep=",\s*", engine="python")
csv_data = df.to_csv(index=False)
csv_data

'TABLE_NAME,FIELD_NAME,DESCRIPTION,DATA_TYPE\ndummy_table,Name,The name of the person,TEXT\ndummy_table,Age,The age of the person,INTEGER\ndummy_table,Gender,The gender of the person,TEXT\ndummy_table,Income,The income of the person,REAL\ndummy_table,ozip,The origin zip code,REAL\ndummy_table,dzip,The destination zip code,REAL\ndummy_table,signup,The date of signup,TEXT\n'

In [130]:
def_list = rsp.split('\n')
def_list

['TABLE_NAME, FIELD_NAME, DESCRIPTION, DATA_TYPE',
 'dummy_table, Name, The name of the person, TEXT',
 'dummy_table, Age, The age of the person, INTEGER',
 'dummy_table, Gender, The gender of the person, TEXT',
 'dummy_table, Income, The income of the person, REAL',
 'dummy_table, ozip, The origin zip code, REAL',
 'dummy_table, dzip, The destination zip code, REAL',
 'dummy_table, signup, The date of signup, TEXT']

In [121]:
for d in def_list:
    print(d.strip("'"))

dummy_table, Name, The name of the person, TEXT
dummy_table, Age, The age of the person, INTEGER
dummy_table, Gender, The gender of the person, TEXT
dummy_table, Income, The income of the person, REAL
dummy_table, ozip, The origin zip code, REAL
dummy_table, dzip, The destination zip code, REAL
dummy_table, signup, The date of signup, TEXT
